# 🎬 YouTube Video Transcription + AI Summary - One-Click Setup

**Super simple - just 2 steps:**

1. **Run the cell below** ⬇️ (takes 2-3 minutes to prepare)
2. **Paste your YouTube URL** when prompted
3. **That's it!** Transcript + AI Summary auto-downloads when done

Works with any YouTube video URL (even long ones!)

✨ **NEW:** Automatic AI-powered summary of your transcript!

💡 **Tip:** Go to Runtime → Change runtime type → GPU (optional, makes it faster)

In [ ]:
#!/usr/bin/env python3
"""
ONE-STEP YOUTUBE TRANSCRIPTION + AI SUMMARY
Setup + Download + Transcribe + Summarize + Save (automatic)
"""

print("\n" + "="*70)
print("🎬  INSTALLING & SETTING UP YOUTUBE TRANSCRIPTION (ONE TIME)")
print("="*70 + "\n")

# Step 1: Install dependencies
print("📦 Installing dependencies...")
import subprocess
subprocess.run(['pip', 'install', '-q', 'openai-whisper'], check=True)
subprocess.run(['pip', 'install', '-q', 'yt-dlp'], check=True)
subprocess.run(['pip', 'install', '-q', 'groq'], check=True)  # Free LLM API
subprocess.run(['apt-get', '-qq', 'install', '-y', 'ffmpeg'], check=True)
print("✓ Dependencies installed\n")

# Step 2: Import libraries
print("📚 Importing libraries...")
import whisper
import os
from pathlib import Path
from google.colab import files
import torch
import re
from groq import Groq
print("✓ Libraries ready\n")

# Step 3: Load model
print("🤖 Loading Whisper Large model...")
print("   (This takes ~1-2 minutes first time)\n")
model = whisper.load_model("large")
print("✓ Model loaded\n")

# Step 4: Get API Key (optional for summary)
print("="*70)
print("🤖 AI SUMMARY SETUP (OPTIONAL)")
print("="*70)
print("To get AI summary, you need a FREE Groq API key:")
print("1. Visit: https://console.groq.com/keys")
print("2. Sign up (free)")
print("3. Create an API key")
print("4. Paste it below (or press Enter to skip summary)\n")

groq_api_key = input("Groq API Key (or press Enter to skip): ").strip()
use_summary = bool(groq_api_key)

if use_summary:
    print("✓ AI Summary enabled\n")
else:
    print("⚠️  Skipping AI summary (you can still get the transcript)\n")

# Step 5: Get YouTube URL
print("="*70)
print("🔗 ENTER YOUTUBE URL")
print("="*70)
youtube_url = input("Paste YouTube URL here: ").strip()

if not youtube_url:
    print("⚠️  No URL provided. Please try again.")
else:
    print(f"\n✓ URL received: {youtube_url}\n")
    
    # Step 6: Download audio from YouTube
    print("="*70)
    print("📥 DOWNLOADING AUDIO FROM YOUTUBE")
    print("="*70 + "\n")
    
    audio_file = "youtube_audio.mp3"
    
    try:
        result = subprocess.run([
            'yt-dlp',
            '-x',  # Extract audio
            '--audio-format', 'mp3',
            '--audio-quality', '0',  # Best quality
            '-o', audio_file.replace('.mp3', '.%(ext)s'),
            '--no-playlist',  # Only download single video
            youtube_url
        ], check=True, capture_output=True, text=True)
        
        print(f"✓ Audio downloaded: {audio_file}\n")
        
        # Step 7: Transcribe
        print("="*70)
        print("🎙️  TRANSCRIBING (wait 2-5 minutes depending on length)")
        print("="*70 + "\n")
        
        def format_timestamp(seconds):
            hours = int(seconds // 3600)
            minutes = int((seconds % 3600) // 60)
            secs = int(seconds % 60)
            return f"{hours:02d}:{minutes:02d}:{secs:02d}"
        
        print(f"📝 Processing: {audio_file}")
        print("-" * 70)
        
        # Transcribe (auto-detect language)
        result = model.transcribe(
            audio_file,
            task='transcribe',
            verbose=False
        )
        
        detected_language = result['language']
        print(f"✓ Detected language: {detected_language}\n")
        
        transcript_text = result['text'].strip()
        
        # Step 8: Generate AI Summary (if enabled)
        summary_text = ""
        if use_summary and transcript_text:
            print("="*70)
            print("🧠 GENERATING AI SUMMARY (wait ~30 seconds)")
            print("="*70 + "\n")
            
            try:
                client = Groq(api_key=groq_api_key)
                
                # Split long transcripts to avoid token limits
                max_chars = 15000
                transcript_to_summarize = transcript_text[:max_chars]
                if len(transcript_text) > max_chars:
                    transcript_to_summarize += "... (transcript truncated for summary)"
                
                chat_completion = client.chat.completions.create(
                    messages=[
                        {
                            "role": "system",
                            "content": "You are a helpful assistant that creates clear, concise summaries of video transcripts. Provide: 1) A brief overview (2-3 sentences), 2) Key points (bullet points), 3) Main takeaways."
                        },
                        {
                            "role": "user",
                            "content": f"Please summarize this video transcript:\n\n{transcript_to_summarize}"
                        }
                    ],
                    model="llama-3.3-70b-versatile",  # Fast and free
                    temperature=0.3,
                    max_tokens=1000
                )
                
                summary_text = chat_completion.choices[0].message.content
                print("✓ AI Summary generated\n")
                
                # Show summary preview
                print("📋 Summary Preview:")
                print("-" * 70)
                print(summary_text[:500])
                if len(summary_text) > 500:
                    print("...(more)")
                print("-" * 70 + "\n")
                
            except Exception as e:
                print(f"⚠️  Summary generation failed: {e}")
                print("Continuing with transcript only...\n")
                summary_text = ""
        
        # Step 9: Save transcript (with summary if available)
        print("="*70)
        print("💾 SAVING TRANSCRIPT" + (" + SUMMARY" if summary_text else ""))
        print("="*70 + "\n")
        
        # Create safe filename from URL
        video_id_match = re.search(r'(?:v=|youtu\.be/)([^&]+)', youtube_url)
        video_id = video_id_match.group(1) if video_id_match else 'youtube'
        transcript_path = f'youtube_{video_id}_transcript.txt'
        
        with open(transcript_path, 'w', encoding='utf-8') as f:
            f.write(f"YouTube Transcript\n")
            f.write(f"URL: {youtube_url}\n")
            f.write(f"Language: {detected_language}\n")
            f.write("=" * 70 + "\n\n")
            
            # Add AI Summary section if available
            if summary_text:
                f.write("🧠 AI SUMMARY\n")
                f.write("=" * 70 + "\n")
                f.write(summary_text)
                f.write("\n\n" + "=" * 70 + "\n\n")
            
            f.write("📝 FULL TRANSCRIPT\n")
            f.write("=" * 70 + "\n")
            f.write(transcript_text)
            f.write("\n\n" + "=" * 70 + "\n")
            f.write("⏱️  DETAILED SEGMENTS\n")
            f.write("=" * 70 + "\n\n")
            
            for segment in result['segments']:
                start = format_timestamp(segment['start'])
                end = format_timestamp(segment['end'])
                text = segment['text'].strip()
                f.write(f"[{start} → {end}] {text}\n")
        
        print(f"✓ File created: {transcript_path}\n")
        
        # Show transcript preview
        print("📋 Transcript Preview:")
        print("-" * 70)
        preview = transcript_text[:300]
        print(preview)
        if len(transcript_text) > 300:
            print("...(more)")
        print("-" * 70)
        
        # Step 10: Auto-download
        print(f"\n📥 Downloading transcript...")
        files.download(transcript_path)
        print(f"✓ Downloaded: {transcript_path}\n")
        
        # Cleanup
        if os.path.exists(audio_file):
            os.remove(audio_file)
            print("✓ Cleaned up temporary audio file\n")
        
    except subprocess.CalledProcessError as e:
        print(f"❌ Error downloading video: {e}")
        print("Make sure the URL is valid and the video is accessible.")
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "="*70)
print("🎉 ALL DONE!")
print("="*70)
print("Your transcript" + (" + AI summary" if use_summary else "") + " is downloaded to ~/Downloads/\n")
print("Next step: Run this in terminal to organize:")
print("   python3 scripts/trigger_colab.py --organize")
print("="*70 + "\n")

## ✨ That's it!

Your transcript (+ AI summary if you provided an API key) is ready and automatically downloaded.

### 🤖 About the AI Summary (Optional)

The notebook uses **Groq** for fast, free AI summaries:
- **Model:** Llama 3.3 70B (very smart!)
- **Speed:** ~30 seconds
- **Free tier:** Generous limits
- **Get your key:** [console.groq.com/keys](https://console.groq.com/keys)

The summary includes:
- 📋 Brief overview
- 🎯 Key points (bullet format)  
- 💡 Main takeaways

### 📁 Organize Your Files

**Back in terminal, run:**
```
python3 scripts/trigger_colab.py --organize
```

This moves your transcript to the `project/transcripts/` folder. Done! 🎉